# Example notebook on extracting satellite data from NASA

* Pre-requirement: you will need to create an account here: https://urs.earthdata.nasa.gov/users/new

In [ ]:
# Install necessary packages
!pip install earthaccess
!pip install folium

In [ ]:
import folium
from folium import plugins

# Xi'an coordinates
xian_lat, xian_lon = 34.3416, 108.9398

# Define the bounding box coordinates for Xi'an
min_lon, min_lat, max_lon, max_lat = 108.7, 34.2, 109.2, 34.5

# Create map centered on Xi'an
m = folium.Map(location=[xian_lat, xian_lon], zoom_start=9)

# Add NASA MODIS layer from GIBS (MODIS Terra true color)
gibs_url = (
    'https://gibs.earthdata.nasa.gov/wmts/epsg3857/best/'
    'MODIS_Terra_CorrectedReflectance_TrueColor/default/'
    '2024-07-01/GoogleMapsCompatible_Level9/{z}/{y}/{x}.jpg'
)

# Add bounding box rectangle
bounding_box = [
    [min_lat, min_lon],  # Southwest corner
    [min_lat, max_lon],  # Southeast corner
    [max_lat, max_lon],  # Northeast corner
    [max_lat, min_lon],  # Northwest corner
    [min_lat, min_lon]   # Close the rectangle
]

folium.PolyLine(
    locations=bounding_box,
    color='red',
    weight=3,
    opacity=0.8,
    popup='MODIS Data Bounding Box',
    tooltip='Study Area Boundary'
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m

In [ ]:
import earthaccess
from datetime import datetime

# Authenticate (you may need to set up credentials)
earthaccess.login()

# Define the bounding box coordinates for Xi'an
min_lon, min_lat, max_lon, max_lat = 108.7, 34.2, 109.2, 34.5

# Set time range
start_date = "2024-01-01"
end_date = "2024-08-01"

# Search for MODIS Land Surface Temperature collection
collections = earthaccess.search_datasets(
    short_name="MOD11A1",
    cloud_hosted=True
)

# Search for available granules (files) over Xi'an
granules = earthaccess.search_data(
    short_name="MOD11A1",
    bounding_box=(min_lon, min_lat, max_lon, max_lat),
    temporal=(start_date, end_date),
    cloud_hosted=True,
    count=100,
)

# Debug: Check the structure of granules
print(f"Found {len(granules)} granules")
if len(granules) > 0:
    print("First granule keys:", granules[0].keys() if hasattr(granules[0], 'keys') else dir(granules[0]))
    print("First granule:", granules[0])

In [ ]:
# Extract available dates
available_dates = sorted(set([g['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime'] for g in granules]))
available_dates

In [ ]:
# Download the granules to local storage
downloaded_files = earthaccess.download(
    granules[:5],  # Download first 5 granules as example
    local_path="./modis_data/",  # Local directory to save files
    provider="POCLOUD"  # Cloud provider (use POCLOUD for cloud-hosted data)
)

print(f"Downloaded {len(downloaded_files)} files:")
for file in downloaded_files:
    print(f"  - {file}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
import os

def extract_lst_from_hdf(hdf_file):
    """
    Extract Land Surface Temperature from MODIS HDF file
    """
    try:
        # Open HDF file
        hdf = SD(hdf_file, SDC.READ)
        
        # Get LST datasets (Day and Night)
        lst_day = hdf.select('LST_Day_1km')
        lst_night = hdf.select('LST_Night_1km')
        
        # Read the data
        lst_day_data = lst_day.get()
        lst_night_data = lst_night.get()
        
        # Get scale and offset attributes
        day_attrs = lst_day.attributes()
        night_attrs = lst_night.attributes()
        
        # Apply scaling: Temperature = DN * scale_factor + add_offset
        day_scale = day_attrs.get('scale_factor', 0.02)
        day_offset = day_attrs.get('add_offset', 0.0)
        night_scale = night_attrs.get('scale_factor', 0.02)  
        night_offset = night_attrs.get('add_offset', 0.0)
        
        # Convert to actual temperature (Kelvin)
        lst_day_temp = lst_day_data * day_scale + day_offset
        lst_night_temp = lst_night_data * night_scale + night_offset
        
        # Convert to Celsius
        lst_day_celsius = lst_day_temp - 273.15
        lst_night_celsius = lst_night_temp - 273.15
        
        # Handle fill values (typically 0 or very high values)
        lst_day_celsius[lst_day_data == 0] = np.nan
        lst_night_celsius[lst_night_data == 0] = np.nan
        
        hdf.end()
        
        return lst_day_celsius, lst_night_celsius
        
    except Exception as e:
        print(f"Error processing {hdf_file}: {e}")
        return None, None

# Process downloaded files
for hdf_file in downloaded_files:
    if hdf_file.endswith('.hdf'):
        print(f"Processing: {hdf_file}")
        day_temp, night_temp = extract_lst_from_hdf(hdf_file)
        
        if day_temp is not None:
            print(f"Day temperature range: {np.nanmin(day_temp):.1f}°C to {np.nanmax(day_temp):.1f}°C")
            print(f"Night temperature range: {np.nanmin(night_temp):.1f}°C to {np.nanmax(night_temp):.1f}°C")

In [ ]:
plt.imshow(night_temp)
plt.colorbar(label='Night LST (°C)')